In [1]:
# Installation des librairies nécessaires
!pip install tensorflow pillow scipy

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import resample
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input

# --- CONFIGURATION DES CHEMINS ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
PATH_IMG_TRAIN = os.path.join(PROJECT_ROOT, "images", "images", "image_train")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")

# Création du dossier de sortie si ça bug
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📂 Racine projet : {PROJECT_ROOT}")
print(f"📂 Images sources : {PATH_IMG_TRAIN}")
print(f"📂 Dossier de sortie (pour Oussama) : {OUTPUT_DIR}")

📂 Racine projet : C:\Users\amisf\Desktop\datascientest_projet
📂 Images sources : C:\Users\amisf\Desktop\datascientest_projet\images\images\image_train
📂 Dossier de sortie (pour Oussama) : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs


C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:


# --- 1. LE BON  CHEMINS ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"

# Chemin spécifique pour les fichiers CSV
PATH_CSV = os.path.join(PROJECT_ROOT, "data", "raw")

# Chemin spécifique pour les images (Mise à jour pour éviter la prochaine erreur)
PATH_IMG_TRAIN = os.path.join(PROJECT_ROOT, "data", "raw", "images", "images", "image_train")

print(f"📂 CSV recherchés dans : {PATH_CSV}")
print(f"📂 Images recherchées dans : {PATH_IMG_TRAIN}")

# --- 2. CHARGEMENT DES DONNÉES ---
try:
    df_x = pd.read_csv(os.path.join(PATH_CSV, "X_train_update.csv"), index_col=0)
    df_y = pd.read_csv(os.path.join(PATH_CSV, "Y_train_CVw08PX.csv"), index_col=0)
    df = pd.merge(df_x, df_y, left_index=True, right_index=True)
    print("✅ Fichiers CSV chargés avec succès.")
except FileNotFoundError as e:
    print(f"❌ Erreur : Impossible de trouver les fichiers dans {PATH_CSV}")
    raise e

# Construction du nom de fichier complet pour Keras
df['filename'] = df.apply(lambda x: f"image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)
df['class_str'] = df['prdtypecode'].astype(str) # Keras demande des strings pour les classes

# --- 3. OVERSAMPLING CIBLÉ À 15 000 ---
TARGET_COUNT = 15000
df_balanced = pd.DataFrame()

print(f"⚖️ Démarrage du rééquilibrage (Cible : {TARGET_COUNT} images par classe)...")

for code in df['prdtypecode'].unique():
    df_class = df[df['prdtypecode'] == code]
    
    # On duplique les données (oversampling) pour atteindre 15 000
    df_resampled = resample(df_class, 
                            replace=True,     
                            n_samples=TARGET_COUNT, 
                            random_state=42)
    df_balanced = pd.concat([df_balanced, df_resampled])

# Mélange final
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Dataset original : {len(df)} images.")
print(f"✅ Dataset augmenté : {len(df_balanced)} images prêt à l'emploi.")

📂 CSV recherchés dans : C:\Users\amisf\Desktop\datascientest_projet\data\raw
📂 Images recherchées dans : C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train
✅ Fichiers CSV chargés avec succès.
⚖️ Démarrage du rééquilibrage (Cible : 15000 images par classe)...
✅ Dataset original : 84916 images.
✅ Dataset augmenté : 405000 images prêt à l'emploi.


In [5]:
# Définition de la taille d'entrée (Standard ResNet = 224x224)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Chargement de ResNet50 sans la tête de classification
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

# Ajout du pooling pour aplatir les résultats
x = base_model.output
x = GlobalAveragePooling2D()(x)
model_extractor = Model(inputs=base_model.input, outputs=x)

print("🧠 Modèle ResNet50 chargé pour l'extraction de features.")
model_extractor.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step 
🧠 Modèle ResNet50 chargé pour l'extraction de features.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [6]:
# Configuration de l'Augmentation
# Ces paramètres seront appliqués à CHAQUE image passant dans le modèle
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Important pour ResNet
    rotation_range=30,      # Rotation aléatoire +/- 30°
    width_shift_range=0.2,  # Décalage horizontal
    height_shift_range=0.2, # Décalage vertical
    shear_range=0.1,        # Cisaillement
    zoom_range=0.2,         # Zoom aléatoire
    horizontal_flip=True,   # Miroir horizontal
    fill_mode='nearest'     # Remplissage des pixels créés
)

# Création du flux de données
# ATTENTION : shuffle=False est CRUCIAL ici pour garder l'ordre des labels !
generator = datagen.flow_from_dataframe(
    dataframe=df_balanced,
    directory=PATH_IMG_TRAIN,
    x_col="filename",
    y_col="class_str",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None, # On ne veut que les features, pas les labels pour l'instant
    shuffle=False    # IMPÉRATIF pour aligner X et y
)

Found 405000 validated image filenames.


In [7]:
print("🚀 Début de l'extraction des features... Prenez un café ☕ ça peut durer")

# Extraction (Prediction)
# Le résultat sera une matrice Numpy de taille (405000, 2048)
features = model_extractor.predict(generator, verbose=1)

print(f"✅ Extraction terminée. Dimensions des features : {features.shape}")

🚀 Début de l'extraction des features... Prenez un café ☕ ça peut durer
12657/12657 ━━━━━━━━━━━━━━━━━━━━ 9102s 719ms/step  
✅ Extraction terminée. Dimensions des features : (405000, 2048)


In [9]:
import json

# correction : on recupere les labels directement depuis le dataframe
# car le generateur ne les a pas stockes a cause de class_mode=None
# comme shuffle=False, l'ordre est garanti identique

print("🔄 récupération des labels depuis le dataframe...")

# 1. on prend directement la colonne des codes produits
y_train_augmented = df_balanced['prdtypecode'].values

# 2. on recrée le mapping pour les métadonnées (utile pour oussama)
unique_codes = sorted(df_balanced['prdtypecode'].unique())
class_indices_map = {str(code): i for i, code in enumerate(unique_codes)}

# 3. sauvegarde
print(f"💾 sauvegarde en cours dans {OUTPUT_DIR}...")

# sauvegarde des features (ta matrice features existe toujours en mémoire, on ne la perd pas)
np.save(os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy'), features)

# sauvegarde des labels
np.save(os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy'), y_train_augmented)

# sauvegarde des métadonnées
metadata = {
    'num_classes': 27,
    'feature_dim': 2048,
    'model_source': 'resnet50_imagenet_augmented',
    'class_mapping': class_indices_map
}

with open(os.path.join(OUTPUT_DIR, 'metadata_augmented.json'), 'w') as f:
    json.dump(metadata, f)

print("✅ tout est sauvegardé !")
print(f"   -> x (features) : {os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy')}")
print(f"   -> y (labels)   : {os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy')}")

🔄 récupération des labels depuis le dataframe...
💾 sauvegarde en cours dans C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs...
✅ tout est sauvegardé !
   -> x (features) : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs\train_features_resnet50_augmented.npy
   -> y (labels)   : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs\train_labels_augmented.npy


In [10]:
# derniere etape xtraction test
# obj : creer fich pr oussama

print("🚀 démarrage du traitement du jeu de test (X_test)...")

# config chmn test
# dossier img test meme niv que train
PATH_IMG_TEST = os.path.join(PROJECT_ROOT, "data", "raw", "images", "images", "image_test")
print(f"📂 images test recherchées dans : {PATH_IMG_TEST}")

# chrgmnt csv test
try:
    df_test = pd.read_csv(os.path.join(PATH_CSV, "X_test_update.csv"), index_col=0)
    print(f"✅ fichier X_test chargé : {len(df_test)} lignes.")
except FileNotFoundError:
    print(f"❌ erreur : impossible de trouver X_test_update.csv dans {PATH_CSV}")
    raise

# constr noms fich (idem train)
df_test['filename'] = df_test.apply(lambda x: f"image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)

# gen neutre (juste resize pas zoom)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=PATH_IMG_TEST,
    x_col="filename",
    y_col=None,          # pas label pr test
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,     # juste features
    shuffle=False        # imperatif pr ordre ids
)

# xtraction rapide (pas augm)
print("📸 extraction des features test en cours...")
features_test = model_extractor.predict(test_generator, verbose=1)

# svg
print(f"💾 sauvegarde des features test...")
np.save(os.path.join(OUTPUT_DIR, 'test_features_resnet50.npy'), features_test)

# svg ids pr ordre (utile oussama)
np.save(os.path.join(OUTPUT_DIR, 'test_ids.npy'), df_test.index.values)

print("✅ mission accomplie ! tous les fichiers sont prêts.")
print(f"   -> test features : {os.path.join(OUTPUT_DIR, 'test_features_resnet50.npy')}")

🚀 démarrage du traitement du jeu de test (X_test)...
📂 images test recherchées dans : C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_test
✅ fichier X_test chargé : 13812 lignes.
Found 13812 validated image filenames.
📸 extraction des features test en cours...
432/432 ━━━━━━━━━━━━━━━━━━━━ 307s 711ms/step
💾 sauvegarde des features test...
✅ mission accomplie ! tous les fichiers sont prêts.
   -> test features : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs\test_features_resnet50.npy
